In [1]:
pip install linkify-it-py -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import re
import time

from linkify_it import LinkifyIt
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
from tqdm import tqdm
import requests
import xmltodict

%reload_ext autoreload
%autoreload 2
sys.path.append('../scripts')
import pbmd_tools as pbmd

In [2]:
pbmd.read_tokens("../.env")
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")
PUBMED_TOKEN = os.environ.get("PUBMED_TOKEN")

In [3]:
linkify = (
    LinkifyIt()
    .set({"fuzzy_email": False}) 
)

In [4]:
PMIDs = []
queries = []
for year in range(2009, 2023):
    queries.append(f'(("http"[Title/Abstract])) AND (("{year}/01/01"[Date - Publication] : "{year}/12/31"[Date - Publication]))')
stats = pbmd.get_forges_stat(queries, PMIDs)
len(PMIDs)

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:46<00:00,  3.36s/it]


47465

In [10]:
#links_stat = {}

for PMID in tqdm(PMIDs[22192:]):
    try:
        summary = pbmd.get_summary(PMID, PUBMED_TOKEN, "../data/log_files/status_all_links.txt")
    except:
        try:
            summary = pbmd.get_summary(PMID, PUBMED_TOKEN, "../data/log_files/status_all_links.txt")
        except:
            continue
            
    abstract = pbmd.get_abstract_from_summary(summary, "../data/log_files/status_all_links.txt")
    
    if abstract != None:
    
        if linkify.test(abstract):
            for match in linkify.match(abstract):
                link = match.url
                try:
                    key = link.split('/')[2]
                except:
                    print(link)
                    continue
                if key in links_stat:
                    links_stat[key] += 1
                else:
                    links_stat[key] = 1

100%|██████████████████████████████████████████████████████████████████████████| 25273/25273 [5:20:40<00:00,  1.31it/s]


In [11]:
with open("../data/links_stat.json", "w") as f:
    json.dump(links_stat, f)

In [22]:
sum(links_stat.values())

56629

In [23]:
len(links_stat)

17618

In [24]:
links_stat_lower = {}

for key in links_stat.keys():
    if key.lower() in links_stat_lower:
        links_stat_lower[key.lower()] += links_stat[key]
    else:
        links_stat_lower[key.lower()] = links_stat[key]

In [25]:
len(links_stat_lower)

17425

In [26]:
for key in links_stat_lower.keys():
    if key.startswith('www'):
        if key[4:] in links_stat_lower:
            links_stat_lower[key[4:]] += links_stat_lower[key]
            links_stat_lower[key] = 0

In [27]:
keys_to_delete = [key for key, value in links_stat_lower.items() if value == 0]
for key in keys_to_delete:
    del links_stat_lower[key]

In [28]:
len(links_stat_lower)

16932

In [29]:
with open("../data/links_stat_lower.json", "w") as f:
    json.dump(links_stat_lower, f)

In [ ]:
'github.com': 1147
'sourceforge.net': 352
'code.google.com': 206
'bitbucket.org': 45
'gitlab.com': 16

In [32]:
dict(sorted(links_stat_lower.items(), key=lambda x: x[1], reverse=True))

{'clinicaltrials.gov': 6087,
 'links.lww.com': 2327,
 'dx.doi.org': 1986,
 'webcitation.org': 1215,
 'github.com': 1147,
 'chictr.org.cn': 1115,
 'www.elsevier.com': 858,
 'onlinelibrary.wiley.com': 772,
 'www.diagnosticpathology.diagnomx.eu': 639,
 'ncbi.nlm.nih.gov': 493,
 'anzctr.org.au': 457,
 'isrctn.com': 448,
 'controlled-trials.com': 432,
 'ow.ly': 368,
 'sourceforge.net': 352,
 'trialregister.nl': 342,
 'www.crd.york.ac.uk': 336,
 'bioconductor.org': 320,
 'umin.ac.jp': 269,
 'www.biomedcentral.com': 259,
 'cran.r-project.org': 253,
 'cancerres.aacrjournals.org': 251,
 'doi.org': 244,
 'www.ebi.ac.uk': 221,
 'code.google.com': 206,
 'bit.ly': 195,
 'irct.ir': 188,
 'youtu.be': 185,
 'proteomecentral.proteomexchange.org': 173,
 'rimed.org': 172,
 'radiology.rsna.org': 146,
 'www.springer.com': 142,
 'www.cdc.gov': 124,
 'ctri.nic.in': 123,
 'journals.sagepub.com': 111,
 'who.int': 99,
 'ensaiosclinicos.gov.br': 99,
 'apps.who.int': 93,
 'nt.ars-grin.gov': 92,
 'chictr.org': 88,